In [1]:
import pygmt
import numpy as np
import pandas as pd
import geopy.distance

In [2]:
def pgd(
        mag_w,
        distance_km,
        *,
        depth=0,
        method='Melgar2D_2015'):
    """
    pgd Calculates expected peak ground displacment based on hypo-central
    distance and Mw. PGD is defined as sqrt(x^2+y^2+z^2) for all methods
    except Melgar2D_2015, where it is sqrt(x^2+y^2).
    Distance units are in km.
    pgd is returned in cm.

    Parameters
    ----------
    :param mag_w: float
        Magnitude of earthquake
    :param distance_km: float or list of floats
        distance from earthquake, assumed to be epicentral distance unless a
        depth is supplied.
    :param depth: float
        optional
        depth of earthquake in km, depth positive, default is 0 km
    :param method: string
        optional
        method used for pgd calculation, can be
        Melgar2D_2015 (default)
        Melgar3D_2015
        Crowell_2016
        Ruhl_2019 (Do not use...)
    :return:
        float or list of floats (if distance is supplied as an list), output is in cm.

    UNAVCO 2020-2022 (Mencin)
    """
    distance = np.array(distance_km)

    if method == 'Melgar2D_2015':
        a = -4.639
        b = 1.063
        c = -0.137
    elif method == 'Melgar3D_2015':
        a = -4.434
        b = 1.047
        c = -0.138
    elif method == 'Crowell_2016':
        a = -6.687
        b = 1.500
        c = -0.214
    elif method == 'Ruhl_2019':
        a = -5.919
        b = 1.009
        c = -0.145
        distance_km = distance_km*1000
    else:
        sys.exit("Invalid method, use: Melgar2D_2015 (default), Melgar3D_2015, Crowell_2016 or Ruhl_2019.")

    log_pgd = a + b * mag_w + c * mag_w * np.log10(np.sqrt(np.power(distance, 2) + np.power(depth, 2)))
    peak_ground_displacement_mm = np.float_power(10, log_pgd)

    if method == 'Ruhl_2019':
        peak_ground_displacement_mm = peak_ground_displacement_mm * 100
    else:
        peak_ground_displacement_mm = peak_ground_displacement_mm

    return peak_ground_displacement_mm

In [3]:
colombia_stations = 'Colombia-Noise.csv'
ecuador_stations = 'Ecuador-Noise.csv'
earthquakes_file = 'Earthquakes.csv'
cities_file = 'Cities.csv'

In [4]:
# Read in station locations and noise for each station
dfE = pd.read_csv(ecuador_stations)
dfC = pd.read_csv(colombia_stations)
frames = [dfC, dfE ]
df = pd.concat(frames)

In [5]:
cities = pd.read_csv(cities_file)

In [6]:
eqs = pd.read_csv(earthquakes_file)
eqs['Date'] = pd.to_datetime(eqs['Date'])

In [7]:
#minimum number of stations for detection
min = 4
#Seimic velocity in km/sec
vel = 4
# Time required to process and issue warning
issue = 6

In [8]:
cities

,Latitude,Longitude,City
0,4.711000,-74.072100,Bogota
1,6.247600,-75.565800,Medellin
2,3.451600,-76.532000,Cali
3,11.004100,-74.807000,Barranquilla
4,10.393200,-75.483200,Cartagena
5,-2.190000,-79.887500,Guayaquil
6,0.220000,-78.512500,Quito
7,0.966667,-79.652778,Esmeraldas
8,0.362678,-78.130667,Ibarra
9,0.811667,-77.718611,Tulcan


In [9]:
# Make a table that shows the expected PGD in each city from each earthquake
pgds = eqs.copy()

for i in cities.itertuples():
    pgds[i[3]+'_PGD cm'] = eqs.apply(lambda row: pgd(row.Mw,
                                     geopy.distance.geodesic( [i[1], i[2]],(row.Latitude,row.Longitude)).km),
                                     axis=1)
pgds       

,Latitude,Longitude,Depth,Date,Mw,Bogota_PGD cm,Medellin_PGD cm,Cali_PGD cm,Barranquilla_PGD cm,Cartagena_PGD cm,Guayaquil_PGD cm,Quito_PGD cm,Esmeraldas_PGD cm,Ibarra_PGD cm,Tulcan_PGD cm
0,4.800,-77.180,19.1,1991-11-19,7.2,3.246280,4.641924,6.696227,1.540635,1.747456,1.366062,2.134340,2.227935,2.242298,2.524613
1,4.720,-77.570,16.0,2004-11-15,7.2,2.889964,4.001353,6.114357,1.491579,1.690577,1.404703,2.213630,2.366398,2.315512,2.597016
2,0.955,-79.369,25.0,2006-01-31,8.8,18.628978,18.591910,35.786679,9.869482,10.842945,44.108828,153.531872,807.568502,121.024253,96.350092
3,0.025,-79.955,25.0,2042-05-14,7.8,3.392828,3.354778,5.432202,2.043594,2.205222,12.567949,19.554987,29.744333,15.090620,11.619666
4,1.000,-79.400,60.0,2058-01-19,7.8,3.969284,3.973861,7.091090,2.266372,2.464401,8.409540,24.512200,125.830578,20.101914,16.652800
5,0.380,-79.950,20.0,2016-04-16,7.7,3.004805,3.000654,4.898114,1.816812,1.963593,9.071345,16.524746,38.160387,12.971187,10.260746
6,2.320,-78.810,19.7,1979-12-12,8.1,7.893155,8.280664,17.841305,4.124305,4.561166,9.203853,21.929967,30.047787,22.493440,25.285194


In [10]:
pgds.to_csv('PGDTable.csv', float_format="%.1f")

In [11]:
def blind_spot(
        eq_lat,
        eq_lon,
        city_lat,
        city_lon,
        df,
        mini):
    df['distance']   = df.apply(lambda row: geopy.distance.geodesic((eq_lat,eq_lon),(row.lat,row.lon)).km, axis=1)
    df.sort_values(by='distance', inplace=True, ignore_index=True)
    dect_dist = df.loc[mini-1,'distance']
    return dect_dist

In [12]:
def warning_time(
        eq_lat,
        eq_lon,
        city_lat,
        city_lon,
        df,
        mini,
        velocity,
        detect_t):
    df['distance']   = df.apply(lambda row: geopy.distance.geodesic((eq_lat,eq_lon),(row.lat,row.lon)).km, axis=1)
    df.sort_values(by='distance', inplace=True, ignore_index=True)
    dect_dist = df.loc[mini-1,'distance']
    city_dist = geopy.distance.geodesic((eq_lat,eq_lon),(city_lat,city_lon)).km
    warning_time_sec = ((city_dist - dect_dist)/velocity) - detect_t
    return warning_time_sec

In [13]:
# make a table for the blind spot diameter (the blind spot is the diameter of first detcion at a specifc minium)
# these should be city independent
blind = eqs.copy()
for i in cities.itertuples():
    blind[i[3]+'_BlindSpot km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)
blind

,Latitude,Longitude,Depth,Date,Mw,Bogota_BlindSpot km,Medellin_BlindSpot km,Cali_BlindSpot km,Barranquilla_BlindSpot km,Cartagena_BlindSpot km,Guayaquil_BlindSpot km,Quito_BlindSpot km,Esmeraldas_BlindSpot km,Ibarra_BlindSpot km,Tulcan_BlindSpot km
0,4.800,-77.180,19.1,1991-11-19,7.2,127.368726,127.368726,127.368726,127.368726,127.368726,127.368726,127.368726,127.368726,127.368726,127.368726
1,4.720,-77.570,16.0,2004-11-15,7.2,152.377377,152.377377,152.377377,152.377377,152.377377,152.377377,152.377377,152.377377,152.377377,152.377377
2,0.955,-79.369,25.0,2006-01-31,8.8,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788
3,0.025,-79.955,25.0,2042-05-14,7.8,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923
4,1.000,-79.400,60.0,2058-01-19,7.8,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605
5,0.380,-79.950,20.0,2016-04-16,7.7,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075
6,2.320,-78.810,19.7,1979-12-12,8.1,105.618102,105.618102,105.618102,105.618102,105.618102,105.618102,105.618102,105.618102,105.618102,105.618102


In [14]:
blind.to_csv('BlindSpotTable.csv', float_format="%.1f")

In [15]:
# make a table for the Warning times
warn = eqs.copy()
for i in cities.itertuples():
    warn[i[3]+'_Warning sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel, issue), axis=1)
warn

,Latitude,Longitude,Depth,Date,Mw,Bogota_Warning sec,Medellin_Warning sec,Cali_Warning sec,Barranquilla_Warning sec,Cartagena_Warning sec,Guayaquil_Warning sec,Quito_Warning sec,Esmeraldas_Warning sec,Ibarra_Warning sec,Tulcan_Warning sec
0,4.800,-77.180,19.1,1991-11-19,7.2,48.389544,22.166448,3.547065,145.733910,123.725834,169.537152,94.074762,88.458192,87.638046,73.424131
1,4.720,-77.570,16.0,2004-11-15,7.2,52.924680,25.663665,1.290537,145.603963,122.985762,157.502859,83.033501,74.717126,77.364499,64.027789
2,0.955,-79.369,25.0,2006-01-31,8.8,164.238120,164.535985,88.914322,289.207627,266.306339,72.225076,15.416517,-8.000211,22.249298,30.192093
3,0.025,-79.955,25.0,2042-05-14,7.8,189.569403,191.782945,115.243727,316.218818,293.167386,42.203116,21.448294,8.299522,32.565108,46.869667
4,1.000,-79.400,60.0,2058-01-19,7.8,164.929178,164.735026,89.453639,289.129806,266.183990,74.016830,17.581765,-8.109801,24.268920,31.872129
5,0.380,-79.950,20.0,2016-04-16,7.7,185.231514,185.497179,110.136980,309.021589,285.850307,53.742781,22.925877,0.882191,33.310245,45.909182
6,2.320,-78.810,19.7,1979-12-12,8.1,114.861487,108.636557,38.217280,231.918511,208.992789,95.822351,26.234404,11.745652,24.909027,19.174105


In [16]:
warn.to_csv('WarningTable.csv', float_format="%.1f")

In [17]:
df25 = pd.read_csv('Sim-25km-Noise.csv')
frames = [dfC, dfE, df25]
df = pd.concat(frames)

In [18]:
# make a table for the blind spot diameter (the blind spot is the diameter of first detcion at a specifc minium)
# these should be city independent
blind25 = eqs.copy()
for i in cities.itertuples():
    blind25[i[3]+'_BlindSpot_(25km) km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)
blind25

,Latitude,Longitude,Depth,Date,Mw,Bogota_BlindSpot_(25km) km,Medellin_BlindSpot_(25km) km,Cali_BlindSpot_(25km) km,Barranquilla_BlindSpot_(25km) km,Cartagena_BlindSpot_(25km) km,Guayaquil_BlindSpot_(25km) km,Quito_BlindSpot_(25km) km,Esmeraldas_BlindSpot_(25km) km,Ibarra_BlindSpot_(25km) km,Tulcan_BlindSpot_(25km) km
0,4.800,-77.180,19.1,1991-11-19,7.2,36.858061,36.858061,36.858061,36.858061,36.858061,36.858061,36.858061,36.858061,36.858061,36.858061
1,4.720,-77.570,16.0,2004-11-15,7.2,37.364977,37.364977,37.364977,37.364977,37.364977,37.364977,37.364977,37.364977,37.364977,37.364977
2,0.955,-79.369,25.0,2006-01-31,8.8,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788
3,0.025,-79.955,25.0,2042-05-14,7.8,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923
4,1.000,-79.400,60.0,2058-01-19,7.8,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605
5,0.380,-79.950,20.0,2016-04-16,7.7,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075
6,2.320,-78.810,19.7,1979-12-12,8.1,36.335531,36.335531,36.335531,36.335531,36.335531,36.335531,36.335531,36.335531,36.335531,36.335531


In [19]:
blind25.to_csv('BlindSpot25Table.csv', float_format="%.1f")

In [20]:
# make a table for the Warning times
warn25 = eqs.copy()
for i in cities.itertuples():
    warn25[i[3]+'_Warning_(25km) sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel, issue), axis=1)
warn25

,Latitude,Longitude,Depth,Date,Mw,Bogota_Warning_(25km) sec,Medellin_Warning_(25km) sec,Cali_Warning_(25km) sec,Barranquilla_Warning_(25km) sec,Cartagena_Warning_(25km) sec,Guayaquil_Warning_(25km) sec,Quito_Warning_(25km) sec,Esmeraldas_Warning_(25km) sec,Ibarra_Warning_(25km) sec,Tulcan_Warning_(25km) sec
0,4.800,-77.180,19.1,1991-11-19,7.2,71.017211,44.794114,26.174732,168.361576,146.353501,192.164818,116.702429,111.085858,110.265712,96.051797
1,4.720,-77.570,16.0,2004-11-15,7.2,81.677780,54.416765,30.043637,174.357063,151.738862,186.255959,111.786601,103.470226,106.117599,92.780889
2,0.955,-79.369,25.0,2006-01-31,8.8,164.238120,164.535985,88.914322,289.207627,266.306339,72.225076,15.416517,-8.000211,22.249298,30.192093
3,0.025,-79.955,25.0,2042-05-14,7.8,189.569403,191.782945,115.243727,316.218818,293.167386,42.203116,21.448294,8.299522,32.565108,46.869667
4,1.000,-79.400,60.0,2058-01-19,7.8,164.929178,164.735026,89.453639,289.129806,266.183990,74.016830,17.581765,-8.109801,24.268920,31.872129
5,0.380,-79.950,20.0,2016-04-16,7.7,185.231514,185.497179,110.136980,309.021589,285.850307,53.742781,22.925877,0.882191,33.310245,45.909182
6,2.320,-78.810,19.7,1979-12-12,8.1,132.182130,125.957200,55.537923,249.239154,226.313432,113.142994,43.555047,29.066295,42.229669,36.494748


In [21]:
warn25.to_csv('Warning25Table.csv', float_format="%.1f")

In [22]:
df50 = pd.read_csv('Sim-50km-Noise.csv')
frames = [dfC, dfE, df50]
df = pd.concat(frames)

In [23]:
# make a table for the blind spot diameter (the blind spot is the diameter of first detcion at a specifc minium)
# these should be city independent
blind50 = eqs.copy()
for i in cities.itertuples():
    blind50[i[3]+'_BlindSpot_(50km) km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)
blind50

,Latitude,Longitude,Depth,Date,Mw,Bogota_BlindSpot_(50km) km,Medellin_BlindSpot_(50km) km,Cali_BlindSpot_(50km) km,Barranquilla_BlindSpot_(50km) km,Cartagena_BlindSpot_(50km) km,Guayaquil_BlindSpot_(50km) km,Quito_BlindSpot_(50km) km,Esmeraldas_BlindSpot_(50km) km,Ibarra_BlindSpot_(50km) km,Tulcan_BlindSpot_(50km) km
0,4.800,-77.180,19.1,1991-11-19,7.2,62.851950,62.851950,62.851950,62.851950,62.851950,62.851950,62.851950,62.851950,62.851950,62.851950
1,4.720,-77.570,16.0,2004-11-15,7.2,57.704034,57.704034,57.704034,57.704034,57.704034,57.704034,57.704034,57.704034,57.704034,57.704034
2,0.955,-79.369,25.0,2006-01-31,8.8,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788
3,0.025,-79.955,25.0,2042-05-14,7.8,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923
4,1.000,-79.400,60.0,2058-01-19,7.8,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605
5,0.380,-79.950,20.0,2016-04-16,7.7,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075
6,2.320,-78.810,19.7,1979-12-12,8.1,56.023500,56.023500,56.023500,56.023500,56.023500,56.023500,56.023500,56.023500,56.023500,56.023500


In [24]:
blind50.to_csv('BlindSpot50Table.csv', float_format="%.1f")

In [25]:
# make a table for the Warning times
warn50 = eqs.copy()
for i in cities.itertuples():
    warn50[i[3]+'_Warning_(50km) sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel, issue), axis=1)
warn50

,Latitude,Longitude,Depth,Date,Mw,Bogota_Warning_(50km) sec,Medellin_Warning_(50km) sec,Cali_Warning_(50km) sec,Barranquilla_Warning_(50km) sec,Cartagena_Warning_(50km) sec,Guayaquil_Warning_(50km) sec,Quito_Warning_(50km) sec,Esmeraldas_Warning_(50km) sec,Ibarra_Warning_(50km) sec,Tulcan_Warning_(50km) sec
0,4.800,-77.180,19.1,1991-11-19,7.2,64.518738,38.295642,19.676260,161.863104,139.855028,185.666346,110.203956,104.587386,103.767240,89.553325
1,4.720,-77.570,16.0,2004-11-15,7.2,76.593016,49.332001,24.958872,169.272298,146.654098,181.171194,106.701836,98.385461,101.032835,87.696125
2,0.955,-79.369,25.0,2006-01-31,8.8,164.238120,164.535985,88.914322,289.207627,266.306339,72.225076,15.416517,-8.000211,22.249298,30.192093
3,0.025,-79.955,25.0,2042-05-14,7.8,189.569403,191.782945,115.243727,316.218818,293.167386,42.203116,21.448294,8.299522,32.565108,46.869667
4,1.000,-79.400,60.0,2058-01-19,7.8,164.929178,164.735026,89.453639,289.129806,266.183990,74.016830,17.581765,-8.109801,24.268920,31.872129
5,0.380,-79.950,20.0,2016-04-16,7.7,185.231514,185.497179,110.136980,309.021589,285.850307,53.742781,22.925877,0.882191,33.310245,45.909182
6,2.320,-78.810,19.7,1979-12-12,8.1,127.260137,121.035208,50.615930,244.317161,221.391440,108.221002,38.633054,24.144302,37.307677,31.572756


In [26]:
warn50.to_csv('Warning50Table.csv', float_format="%.1f")

In [27]:
df100 = pd.read_csv('Sim-100km-Noise.csv')
frames = [dfC, dfE, df100]
df = pd.concat(frames)

In [28]:
# make a table for the blind spot diameter (the blind spot is the diameter of first detcion at a specifc minium)
# these should be city independent
blind100 = eqs.copy()
for i in cities.itertuples():
    blind100[i[3]+'_BlindSpot_(100km) km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)
blind100

,Latitude,Longitude,Depth,Date,Mw,Bogota_BlindSpot_(100km) km,Medellin_BlindSpot_(100km) km,Cali_BlindSpot_(100km) km,Barranquilla_BlindSpot_(100km) km,Cartagena_BlindSpot_(100km) km,Guayaquil_BlindSpot_(100km) km,Quito_BlindSpot_(100km) km,Esmeraldas_BlindSpot_(100km) km,Ibarra_BlindSpot_(100km) km,Tulcan_BlindSpot_(100km) km
0,4.800,-77.180,19.1,1991-11-19,7.2,85.427290,85.427290,85.427290,85.427290,85.427290,85.427290,85.427290,85.427290,85.427290,85.427290
1,4.720,-77.570,16.0,2004-11-15,7.2,86.453890,86.453890,86.453890,86.453890,86.453890,86.453890,86.453890,86.453890,86.453890,86.453890
2,0.955,-79.369,25.0,2006-01-31,8.8,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788,39.612788
3,0.025,-79.955,25.0,2042-05-14,7.8,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923,52.225923
4,1.000,-79.400,60.0,2058-01-19,7.8,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605,36.814605
5,0.380,-79.950,20.0,2016-04-16,7.7,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075,45.291075
6,2.320,-78.810,19.7,1979-12-12,8.1,56.062831,56.062831,56.062831,56.062831,56.062831,56.062831,56.062831,56.062831,56.062831,56.062831


In [29]:
blind100.to_csv('BlindSpot100Table.csv', float_format="%.1f")

In [30]:
# make a table for the Warning times
warn100 = eqs.copy()
for i in cities.itertuples():
    warn100[i[3]+'_Warning_(100km) sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel, issue), axis=1)
warn100

,Latitude,Longitude,Depth,Date,Mw,Bogota_Warning_(100km) sec,Medellin_Warning_(100km) sec,Cali_Warning_(100km) sec,Barranquilla_Warning_(100km) sec,Cartagena_Warning_(100km) sec,Guayaquil_Warning_(100km) sec,Quito_Warning_(100km) sec,Esmeraldas_Warning_(100km) sec,Ibarra_Warning_(100km) sec,Tulcan_Warning_(100km) sec
0,4.800,-77.180,19.1,1991-11-19,7.2,58.874903,32.651807,14.032425,156.219269,134.211193,180.022511,104.560122,98.943551,98.123405,83.909490
1,4.720,-77.570,16.0,2004-11-15,7.2,69.405552,42.144537,17.771408,162.084834,139.466634,173.983730,99.514372,91.197997,93.845371,80.508661
2,0.955,-79.369,25.0,2006-01-31,8.8,164.238120,164.535985,88.914322,289.207627,266.306339,72.225076,15.416517,-8.000211,22.249298,30.192093
3,0.025,-79.955,25.0,2042-05-14,7.8,189.569403,191.782945,115.243727,316.218818,293.167386,42.203116,21.448294,8.299522,32.565108,46.869667
4,1.000,-79.400,60.0,2058-01-19,7.8,164.929178,164.735026,89.453639,289.129806,266.183990,74.016830,17.581765,-8.109801,24.268920,31.872129
5,0.380,-79.950,20.0,2016-04-16,7.7,185.231514,185.497179,110.136980,309.021589,285.850307,53.742781,22.925877,0.882191,33.310245,45.909182
6,2.320,-78.810,19.7,1979-12-12,8.1,127.250305,121.025375,50.606097,244.307329,221.381607,108.211169,38.623222,24.134469,37.297844,31.562923


In [31]:
warn100.to_csv('Warning100Table.csv', float_format="%.1f")